# Sokoban

En aquesta pràctica desenvoluparem un agent capaç de resoldre el joc Sokoban. El joc consisteix en un tauler de caselles quadrades en el qual hi ha un robot, un conjunt de caixes i un conjunt de posicions d'emmagatzematge. El robot pot moure's en les quatre direccions cardinals i pot arrossegar una caixa si aquesta es troba en una casella adjacent al robot. L'objectiu del joc és arrossegar totes les caixes a les posicions d'emmagatzematge.

![Sokoban 75%](sokoban.png)

Si no coneixes el joc Sokoban, pots jugar-hi en línia a [Sokoban Online](http://www.sokobanonline.com/play).

## Exercici 1

Implementa la classe `SokobanState` que representa un estat del joc Sokoban. La classe ha de tenir els següents atributs:
- `action`: L'acció que s'ha aplicat per arribar a l'estat. L'estat inicial té l'acció "START".
- `cost`: El cost de l'acció que s'ha aplicat per arribar a l'estat. L'estat inicial té cost 0.
- `parent`: L'estat pare de l'estat actual. L'estat inicial no té estat pare.
- `width`: L'amplada del tauler. Per defecte, el tauler té 5 caselles d'ample.
- `height`: L'altura del tauler. Per defecte, el tauler té 5 caselles d'alt.
- `robots`: La posició dels robots. Per defecte, hi ha un únic robot a la posició (0, 0).
- `boxes`: La posició de les caixes. Per defecte, hi ha una única caixa a la posició (0, 0).
- `storage`: La posició de les posicions d'emmagatzematge. Per defecte, hi ha una única posició d'emmagatzematge a la posició (0, 0).
- `obstacles`: La posició dels obstacles. Per defecte, no hi ha obstacles.
- `__str__`: Representació gràfica de l'estat del problema. Mostra els robots com a 'R', les caixes com a 'B', les posicions d'emmagatzematge com a 'S' i els obstacles com a 'O'. Les parets i els límits del tauler es mostren com a '#'.
- `__hash__`: Retorna un hash de l'estat. Necessari per a poder utilitzar l'estat com a clau d'un diccionari.
- `__lt__`: Retorna si el cost de l'estat actual és menor que el cost d'un altre estat.
- `__eq__`: Retorna si l'estat actual és igual a un altre estat.

In [1]:
from __future__ import annotations

from dataclasses import dataclass
from itertools import chain


@dataclass
class SokobanState:
    action: str = "START"
    cost: int = 0
    parent: 'SokobanState' = None
    width: int = 5
    height: int = 5
    robots: tuple = ((0, 0))
    boxes: set | frozenset | tuple = ((0, 0))
    storage: set | frozenset | tuple = ((0, 0))
    obstacles: set | frozenset | tuple = ()

    def __str__(self):
        """Representació gràfica de l'estat del problema.
        Mostra els robots com a 'R', les caixes com a 'B', les posicions d'emmagatzematge com a 'S' i els obstacles com a 'O'.
        Les parets i els límits del tauler es mostren com a '#'.    
        """

        out = ""
        for y in range(-1, self.height + 1):
            for x in range(-1, self.width + 1):
                if x == -1 or y == -1 or (x, y) in self.obstacles or x == self.width or y == self.height:
                    out += "🚧"
                elif (x, y) in self.robots:
                    out += "🤖"
                elif (x, y) in self.boxes:
                    out += "📦"
                elif (x, y) in self.storage:
                    out += "❌"
                else:
                    out += "⬜"
            out += "\n"
        return out

    def __hash__(self):
        return hash(self.__str__())

    def __lt__(self, other):
        return self.cost < other.cost

    def __eq__(self, other):
        return self.__hash__() == other.__hash__()


## Exercici 2
Utilitzant com a base la classe `Problema`, implementa la classe `ProblemaSokoban` que representa un problema de Sokoban. Hauràs de sobrescriure, com a mínim, els següents mètodes: 
- `accions`: Retorna les accions possibles per a un estat. Les accions es representen com una tupla (robot, direcció).
- `accio`: Retorna l'estat que resulta d'aplicar una acció a un estat.
- `es_resultat`: Retorna si un estat és un estat final. Un estat és final si totes les caixes estan en una posició d'emmagatzematge.

In [3]:
class Problema(object):
    """Aquesta és la classe abstracta per a un problema formal. Una nova àrea crea una subclasse d'aquesta, sobrescrivint `accions` i `accio`, i potser altres mètodes.
    L'heurística per defecte és 0 i el cost d'acció per defecte és 1 per a tots els estats.
    Quan crees una instància d'una subclasse, especifica els estats `inicial` i `final`
    (o proporciona un mètode `es_resultat`) i potser altres arguments de paraula clau per a la subclasse."""

    def __init__(self, inicial=None, final=None, **kwds):
        self.__dict__.update(inicial=inicial, final=final, **kwds)

    def accions(self, state):         raise NotImplementedError

    def accio(self, state, action):   raise NotImplementedError

    def es_resultat(self, state):     return state == self.final

    def cost_accio(self, s, a, s1):   return 1

    def h(self, estat):               return 0

    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.inicial, self.final)

In [4]:
class ProblemaSokoban(Problema):
    """Classe per a un problema de Sokoban. Un problema de Sokoban es defineix per:
    - Els robots
    - Les caixes
    - Les posicions d'emmagatzematge
    
    El problema es resol quan totes les caixes estan en una posició d'emmagatzematge.
    """

    def accions(self, state):
        """Les accions per a un estat són les accions que poden fer els robots.
        Els robots poden moure's en les quatre direccions cardinals.
        Les accions es representen com una tupla (robot, direcció).
        """
        accions = []
        for robot in state.robots:
            for direccio in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                if self.es_moviment_valid(state, robot, direccio):
                    accions.append((robot, direccio))
        return accions

    def caixa_es_moviment_valid(self, state, param, direccio):
        x, y = param
        dx, dy = direccio

        blocked = list(chain(state.obstacles, state.robots, state.boxes))

        if (x + dx, y + dy) in blocked:
            return False

        if (x + dx, y + dy) in state.boxes:
            return False

        if (x + dx < 0 or x + dx >= state.width) or (y + dy < 0 or y + dy >= state.height):
            return False
        return True

    def es_moviment_valid(self, state, robot, direccio):
        """Un moviment és vàlid si el robot es pot moure en la direcció indicada.
        Un robot es pot moure en una direcció si no hi ha cap obstacle en la direcció
        no hi ha cap altre robot en la casella de destí, no hi ha cap caixa en la casella de destí i no se'n sortim del tauler.
        """
        x, y = robot
        dx, dy = direccio

        blocked = list(chain(state.obstacles, state.robots))

        if (x + dx, y + dy) in blocked:
            return False

        if (x + dx, y + dy) in state.boxes:
            if not self.caixa_es_moviment_valid(state, (x + dx, y + dy), direccio):
                return False

        if (x + dx < 0 or x + dx >= state.width) or (y + dy < 0 or y + dy >= state.height):
            return False
        return True

    def accio(self, state, action):
        """Retorna l'estat resultant d'aplicar l'acció a l'estat donat.
        L'acció és una tupla (robot, direcció).
        """
        robot, direccio = action
        x, y = robot
        dx, dy = direccio

        new_robots = list(state.robots)
        new_robots.remove(robot)
        new_robots.append((x + dx, y + dy))
        new_robots = tuple(new_robots)

        if (x + dx, y + dy) in state.boxes:
            new_boxes = list(state.boxes)
            new_boxes.remove((x + dx, y + dy))
            new_boxes.append((x + 2 * dx, y + 2 * dy))
            new_boxes = frozenset(new_boxes)
        else:
            new_boxes = state.boxes

        return SokobanState(action, state.cost + 1, state, state.width, state.height, new_robots, new_boxes,
                            state.storage, state.obstacles)

    def es_resultat(self, state):
        """Un estat és resultat si totes les caixes estan en una posició d'emmagatzematge."""
        return state.boxes == state.storage

    def h(self, state):
        """Heurística per a un estat. Utilitza la distància Manhattan entre les caixes i les posicions d'emmagatzematge."""
        return sum(min(abs(box[0] - storage[0]) + abs(box[1] - storage[1]) for storage in state.storage) for box in
                   state.boxes)


## Exercici 3

Implementa l'algorisme `UCS` per a resoldre un problema de Sokoban. L'algorisme ha de retornar l'estat final, el cost de l'estat final i la ruta per arribar a l'estat final. La ruta és una llista d'estats que comença en l'estat inicial i acaba en l'estat final. Si no es pot arribar a l'estat final, l'algorisme ha de retornar `None` per a tots els valors.

Utilitza l'algorisme `UCS` per a resoldre el següent problema de Sokoban:

In [5]:
inicial = SokobanState(
    "START", 0, None, 5, 5,  # dimensions
    ((2, 1), (2, 3)),  # robots
    frozenset(((1, 1),)),  # boxes
    frozenset(((0, 0),)),  # storage
    frozenset(((1, 0), (2, 0), (3, 0), (1, 4), (2, 4), (3, 4)))  # obstacles
)
soko = ProblemaSokoban(inicial)

## Exercici 4

Pensa una heurística que sigui admissible per a un problema de Sokoban i implementa-la.

Implementa l'algorisme `Greedy` per a resoldre un problema de Sokoban. L'algorisme ha de retornar l'estat final, el cost de l'estat final i la ruta per arribar a l'estat final. La ruta és una llista d'estats que comença en l'estat inicial i acaba en l'estat final. Si no es pot arribar a l'estat final, l'algorisme ha de retornar `None` per a tots els valors.

Utilitza l'algorisme `Greedy` per a resoldre el mateix problema de Sokoban:

In [6]:
inicial = SokobanState(
    "START", 0, None, 5, 5,  # dimensions
    ((2, 1), (2, 3)),  # robots
    frozenset(((1, 1),)),  # boxes
    frozenset(((0, 0),)),  # storage
    frozenset(((1, 0), (2, 0), (3, 0), (1, 4), (2, 4), (3, 4)))  # obstacles
)
soko = ProblemaSokoban(inicial)

In [8]:
import heapq


def greedy(problema: ProblemaSokoban):
    frontera = []
    heapq.heappush(frontera, (0, problema.inicial, []))
    visitats = set()

    i = 0
    while len(frontera) > 0:
        i += 1
        cost, estat, ruta = heapq.heappop(frontera)
        visitats.add(estat)

        if problema.es_resultat(estat):
            print("Estats visitats Greedy:", len(visitats))
            return estat, cost, ruta + [estat]

        for robot, direccio in problema.accions(estat):
            nou_estat = problema.accio(estat, (robot, direccio))
            if nou_estat not in visitats:
                heapq.heappush(frontera, (
                    problema.h(nou_estat),
                    nou_estat,
                    ruta + [estat]))


estat, cost, ruta = greedy(soko)
print("Cost:", len(ruta) - 1)

for estat in ruta:
    print(estat)
    print()

Estats visitats Greedy: 14
Cost: 4
🚧🚧🚧🚧🚧🚧🚧
🚧❌🚧🚧🚧⬜🚧
🚧⬜📦🤖⬜⬜🚧
🚧⬜⬜⬜⬜⬜🚧
🚧⬜⬜🤖⬜⬜🚧
🚧⬜🚧🚧🚧⬜🚧
🚧🚧🚧🚧🚧🚧🚧


🚧🚧🚧🚧🚧🚧🚧
🚧❌🚧🚧🚧⬜🚧
🚧📦🤖⬜⬜⬜🚧
🚧⬜⬜⬜⬜⬜🚧
🚧⬜⬜🤖⬜⬜🚧
🚧⬜🚧🚧🚧⬜🚧
🚧🚧🚧🚧🚧🚧🚧


🚧🚧🚧🚧🚧🚧🚧
🚧❌🚧🚧🚧⬜🚧
🚧📦⬜⬜⬜⬜🚧
🚧⬜🤖⬜⬜⬜🚧
🚧⬜⬜🤖⬜⬜🚧
🚧⬜🚧🚧🚧⬜🚧
🚧🚧🚧🚧🚧🚧🚧


🚧🚧🚧🚧🚧🚧🚧
🚧❌🚧🚧🚧⬜🚧
🚧📦⬜⬜⬜⬜🚧
🚧🤖⬜⬜⬜⬜🚧
🚧⬜⬜🤖⬜⬜🚧
🚧⬜🚧🚧🚧⬜🚧
🚧🚧🚧🚧🚧🚧🚧


🚧🚧🚧🚧🚧🚧🚧
🚧📦🚧🚧🚧⬜🚧
🚧🤖⬜⬜⬜⬜🚧
🚧⬜⬜⬜⬜⬜🚧
🚧⬜⬜🤖⬜⬜🚧
🚧⬜🚧🚧🚧⬜🚧
🚧🚧🚧🚧🚧🚧🚧



## Exercici 5

Implementa l'algorisme `A*` per a assegurar que obtenin la ruta òptima. Compara el tamany de la ruta, els estats visitats i el temps d'execució amb l'algorisme `Greedy` amb els següents problemes de Sokoban:

(Nota: Pots utilitzar la funció `time.time()` per a calcular el temps d'execució d'un algorisme.)

In [46]:
estat_facil = SokobanState(
    "START", 0, None, 5, 5,  # dimensions
    ((2, 1), (2, 3)),  # robots
    frozenset(((1, 1),)),  # boxes
    frozenset(((0, 0),)),  # storage
    frozenset(((1, 0), (2, 0), (3, 0), (1, 4), (2, 4), (3, 4)))  # obstacles
)

soko_facil = ProblemaSokoban(estat_facil)

estat_mitja = SokobanState(
    "START", 0, None, 6, 4,  # dimensions
    ((2, 1), (2, 2)),  # robots
    frozenset(((1, 1), (4, 2))),  # boxes
    frozenset(((2, 1), (2, 2))),  # storage
    frozenset()  # obstacles
)

soko_mitja = ProblemaSokoban(estat_mitja)

estat_dificil = SokobanState(
    "START", 0, None, 6, 6,  # dimensions
    ((0, 0), (0, 2), (0, 4)),  # robots
    frozenset(((1, 0), (1, 2), (1, 4))),  # boxes
    frozenset(((5, 0), (5, 2), (0, 5))),  # storage
    frozenset()  # obstacles
)

soko_dificil = ProblemaSokoban(estat_dificil)

In [47]:
import heapq


def a_star(problema: ProblemaSokoban):
    frontera = []
    heapq.heappush(frontera, (0, 0, problema.inicial, []))
    visitats = set()

    i = 0
    while len(frontera) > 0:
        cost, h, estat, ruta = heapq.heappop(frontera)

        if estat in visitats:
            continue

        visitats.add(estat)
        i += 1

        #if i % 10000 == 0:
        #    print("Estats visitats:", len(visitats), "Estats frontera:", len(frontera))
        #    print(estat)

        if problema.es_resultat(estat):
            print("Estats visitats A*:", len(visitats))
            return estat, cost, ruta + [estat]

        for robot, direccio in problema.accions(estat):
            nou_estat = problema.accio(estat, (robot, direccio))
            if nou_estat not in visitats:
                h = problema.h(nou_estat)
                tie_break = h * (1 + (i / 1000))
                heapq.heappush(frontera, (
                    h + cost + problema.cost_accio(estat, (robot, direccio), nou_estat),
                    tie_break,
                    nou_estat,
                    ruta + [estat]))


In [48]:
# Si els temps d'execució són molt elevats amb A* pots parar l'execució i tornar més endavant, quan farem millores a l'algorisme.

final, cost, ruta = greedy(soko_facil)
print("Cost Greedy:", len(ruta) - 1)

final, cost, ruta = a_star(soko_facil)
print("Cost A*:", len(ruta) - 1)

final, cost, ruta = greedy(soko_mitja)
print("Cost Greedy:", len(ruta) - 1)

final, cost, ruta = a_star(soko_mitja)
print("Cost A*:", len(ruta) - 1)

final, cost, ruta = greedy(soko_dificil)
print("Cost Greedy:", len(ruta) - 1)

final, cost, ruta = a_star(soko_dificil)
print("Cost A*:", len(ruta) - 1)


Estats visitats Greedy: 14
Cost Greedy: 4
Estats visitats A*: 40
Cost A*: 4
Estats visitats Greedy: 493
Cost Greedy: 17
Estats visitats A*: 1266
Cost A*: 10
Estats visitats Greedy: 73
Cost Greedy: 14
Estats visitats A*: 11831
Cost A*: 14


## Exercici 6

Que has observat? Quin algorisme és més eficient en qüestió de temps i estats visitats? Troba la solució òptima sempre? Per què?

## Exercici 7

L'últim exemple pot resultar molt lent. Una de les causes és que, al tindre molt d'espai lliure, l'heurística no és molt informativa. Tenim molts estats que són igual de bons candidats per a ser l'estat final i l'algorisme `A*` ha d'explorar molts d'aquests estats.

Per solucionar aquest problema, podem utilitzar el `tie breaking`. Aquesta tècnica consisteix en afegir un factor de desempat a la funció d'heurística. D'aquesta manera, si dos estats tenen la mateixa heurística, l'algorisme `A*` explorarà primer l'estat amb el factor de desempat més gran. 

Utilitza l'algorisme `A*` millorat per tindre una solució a `soko_dificil`.

In [52]:
def a_star_millorat(problema: ProblemaSokoban):
    frontera = []
    heapq.heappush(frontera, (0, 0, problema.inicial, []))
    visitats = set()

    i = 0
    while len(frontera) > 0:
        cost, h, estat, ruta = heapq.heappop(frontera)

        if estat in visitats:
            continue

        visitats.add(estat)
        i += 1

        #if i % 10000 == 0:
        #    print("Estats visitats:", len(visitats), "Estats frontera:", len(frontera))
        #    print(estat)

        if problema.es_resultat(estat):
            print("Estats visitats A*:", len(visitats))
            return estat, cost, ruta + [estat]

        for robot, direccio in problema.accions(estat):
            nou_estat = problema.accio(estat, (robot, direccio))
            if nou_estat not in visitats:
                h = problema.h(nou_estat)
                tie_break = h * (1 + (i / 1000))
                heapq.heappush(frontera, (
                    h + cost + problema.cost_accio(estat, (robot, direccio), nou_estat),
                    tie_break,
                    nou_estat,
                    ruta + [estat]))

In [54]:
estat, cost, ruta = a_star_millorat(soko_dificil)
print("Cost A* millorat:", len(ruta) - 1)

Estats visitats A*: 11831
Cost A* millorat: 14
